# Podjetje, varianta 2

In [ ]:
import os
from sqlalchemy import Column, DateTime, String, Integer, ForeignKey, func, create_engine
from sqlalchemy.orm import relationship, backref, declarative_base, sessionmaker

In [ ]:
Base = declarative_base()


class Oddelek(Base):
    __tablename__ = 'oddelek'

    id = Column(Integer, primary_key=True)
    naziv = Column(String)

    # Ime razreda lahko podamo kot niz (ker še ni definirano)
    seznam_zaposlenih = relationship('Zaposleni', secondary='povezava', back_populates='seznam_oddelkov')

    def __repr__(self):
        return f"Oddelek[{self.id}, {self.naziv}]"

    
class Zaposleni(Base):
    __tablename__ = 'zaposleni'

    id = Column(Integer, primary_key=True)
    ime = Column(String)
    datum_zaposlitve = Column(DateTime, default=func.now())
    seznam_oddelkov = relationship(Oddelek, secondary='povezava', back_populates='seznam_zaposlenih')

    def __repr__(self):
        return f"Zaposleni[{self.id}, {self.ime}, {self.datum_zaposlitve}]"


class Povezava(Base):
    __tablename__ = 'povezava'

    oddelek_id = Column(Integer, ForeignKey('oddelek.id'), primary_key=True)
    zaposleni_id = Column(Integer, ForeignKey('zaposleni.id'), primary_key=True)

Povežimo se na bazo - če že obstaja, jo bomo najprej pobrisali.

In [ ]:
baza = 'podjetje2.db'
if os.path.exists(baza):
    os.remove(baza)

engine = create_engine(f'sqlite:///{baza}', echo=True)

In [ ]:
Base.metadata.create_all(engine)
DBSessionMaker = sessionmaker(bind=engine)
session = DBSessionMaker()

Ustvarimo dva oddelka in tri zaposlene.

In [ ]:
IT = Oddelek(naziv="IT")
finance = Oddelek(naziv="Finance")
metka = Zaposleni(ime="Metka")
janez = Zaposleni(ime="Janez")
katka = Zaposleni(ime="Katka")

Dodajmo zaposlene v oddelke in zapišimo v bazo.

In [ ]:
katka.seznam_oddelkov.append(finance)
finance.seznam_zaposlenih.append(metka)
janez.seznam_oddelkov.append(IT)

In [ ]:
session.add(IT)
session.add(finance)
session.add(janez)
session.add(metka)
session.add(katka)
session.commit()

Izpišimo Janezove oddelke.

In [ ]:
janez.seznam_oddelkov

Izpišimo zaposlene na oddelku Finance.

In [ ]:
finance.seznam_zaposlenih

Izpišimo zaposlene na oddelku Finance s poizvedbo na oddelkih.

In [ ]:
session.query(Oddelek).filter(Oddelek.naziv == 'Finance').one().seznam_zaposlenih

Izpišimo zaposlene na oddelku Finance s poizvedbo na zaposlenih.

In [ ]:
session.query(Zaposleni).filter(Zaposleni.seznam_oddelkov.any(Oddelek.naziv == 'Finance')).all()